In [1]:
# Imports
import torch
from torch.distributions import Normal,Laplace,StudentT
from dask_jobqueue import SLURMCluster
from distributed import Client
import BD_RKHS

In [2]:
# Args set up
N = 5000
ntrial = 20
flip_prob = 0.05

In [3]:
cluster = SLURMCluster(
    n_workers=0,
    memory="32GB",
    processes=1,
    cores=1,
    scheduler_options={
        "dashboard_address": ":11111",
        "allowed_failures": 10
    },
    job_cpu=1,
    walltime="24:0:0",
)
cluster.adapt(minimum=0, maximum=100)
client = Client(cluster)

futures = []
for seed in range(ntrial):
    f1 = client.submit(BD_RKHS.run_experiment,seed,flip_prob,N)
    futures += [f1]

results = client.gather(futures)

client.close()
cluster.close()

/nfs/ghome/live/danceh/.local/mambaforge/envs/cocycles/lib/python3.12/site-packages/distributed/node.py:182: UserWarning: Port 11111 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 41577 instead
  warnings.warn(
2024-03-18 18:12:38,932 - distributed.deploy.adaptive_core - INFO - Adaptive stop


In [4]:
client.close()
cluster.close()

In [5]:
torch.save(f = "BD_RKHS_results_new_N={0}_trials={1}_flip_prob={2}.pt".format(N,ntrial,flip_prob), obj = results)